In [1]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-mcy3mge4/gym-walk_69ffbb34b4d3470e8eeea9bf22dcac4f
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-mcy3mge4/gym-walk_69ffbb34b4d3470e8eeea9bf22dcac4f
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py) ... done
  Created wheel for gym-walk: filename=gym_walk-0.0.2-py3-none-any.whl size=5377 sha256=8df022e551e75b611226acb762e0f7791ad3a6755fa441a5ed6fff5162df61fd
  Stored in directory: /tmp/pip-ephem-wheel-cache-89v7uup0/wheels/60/02/77/2dd9f31df8d13bc7c014725f4002e29d0fc3ced5e8ac08e1cf
Successfully built gym-walk


In [54]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

In [55]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [56]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [84]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123)
    results = []
    for _ in range(n_episodes):
        state_info = env.reset(seed=123)
        # Ensure state is an integer
        if isinstance(state_info, tuple):
            state = state_info[0]
        else:
            state = state_info

        if isinstance(state, np.ndarray):
             state = int(state)


        # Modified line to unpack 5 values from env.step
        terminated, truncated, steps = False, False, 0
        while not (terminated or truncated) and steps < max_steps:
            # Access action using indexing instead of calling pi as a function
            action = pi[state] # state is now guaranteed to be an integer
            next_state_info, _, terminated, truncated, h = env.step(action)

            # Ensure next_state is an integer
            if isinstance(next_state_info, tuple):
                next_state = next_state_info[0]
            else:
                next_state = next_state_info

            if isinstance(next_state, np.ndarray):
                next_state = int(next_state)

            state = next_state
            steps += 1
        # The loop condition already handles whether the episode is done (terminated or truncated)
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [88]:
import numpy as np
from tqdm import tqdm
from itertools import count
import gym # Import gym

def mean_return(env, pi, n_episodes=100, max_steps=200):
    results = []
    for _ in range(n_episodes):
        # Handle potential tuple return from reset in newer Gym versions
        state_info = env.reset(seed=123)
        if isinstance(state_info, tuple):
            state = state_info[0]
        else:
            state = state_info

        # Ensure state is an integer for environments that might return an array
        if isinstance(state, np.ndarray):
             state = int(state)

        done, steps = False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            # Get the action from the policy based on the current state
            # Ensure state is treated as an integer index
            action = pi[int(state)] # Access the action from the pi array using the integer state as index
            # Updated line to unpack 4 values from env.step for older Gym versions
            # Assuming the environment in use (FrozenLake-v1) returns 4 values from step
            next_state, reward, terminated, truncated, _ = env.step(action)

            # Calculate the combined 'done' flag
            done = terminated or truncated

            # Ensure next_state is an integer if it's an array or tuple
            if isinstance(next_state, np.ndarray):
                next_state = int(next_state)
            elif isinstance(next_state, tuple):
                next_state = int(next_state[0])

            results[-1] += reward
            steps += 1
            state = next_state # Update the state for the next step
    return np.mean(results)



In [89]:
env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
goal_state = 15
#LEFT, RIGHT = range(2)

In [90]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [91]:
def decay_schedule(
    init_value, min_value, decay_ratio,
    max_steps, log_start = -2, log_base=10):

    decay_steps = int(max_steps * decay_ratio)
    rem_steps = max_steps - decay_steps

    values = np.logspace(
        log_start, 0, decay_steps,
        base=log_base, endpoint=True)[::-1]

    values = (values - values.min()) / (values.max() - values.min())
    values = (init_value - min_value) * values + min_value
    values = np.pad(values, (0, rem_steps), 'edge')
    return values

In [92]:
from itertools import count
import numpy as np
from tqdm import tqdm

In [93]:
import numpy as np
from tqdm import tqdm
from itertools import count

def generate_trajectory(
    select_action, Q, epsilon,
    env, max_steps=200):
  trajectory = []

  state = env.reset()
  # Handle potential tuple return from reset in newer Gym versions
  if isinstance(state, tuple):
      state = state[0]

  # Ensure state is an integer for environments that might return an array
  if isinstance(state, np.ndarray):
      state = int(state)

  for t in count():
      action = select_action(state, Q, epsilon)

      # Update to unpack 5 values from env.step
      next_state, reward, terminated, truncated, _ = env.step(action)

      # Calculate the combined 'done' flag
      done = terminated or truncated

      # Ensure next_state is an integer if it's an array or tuple
      if isinstance(next_state, np.ndarray):
          next_state = int(next_state)
      elif isinstance(next_state, tuple):
          next_state = int(next_state[0])


      experience = (state, action, reward, next_state, done)
      trajectory.append(experience)

      if done:
          break
      if t >= max_steps - 1:
          # If max_steps is reached, consider this trajectory complete
          break
      state = next_state

  return np.array(trajectory,object)

In [94]:
import numpy as np
from tqdm import tqdm
from itertools import count

def mc_control(env,
               gamma=1.0,
               init_alpha=0.5,
               min_alpha=0.01,
               alpha_decay_ratio=0.5,
               init_epsilon=1.0,
               min_epsilon=0.1,
               epsilon_decay_ratio=0.9,
               n_episodes=3000,
               max_steps=200,
               first_visit=True):

    nS, nA = env.observation_space.n, env.action_space.n

    discounts = np.logspace(
        0, max_steps,
        num=max_steps, base=gamma,
        endpoint=False)

    alphas = decay_schedule(
        init_alpha, min_alpha,
        alpha_decay_ratio,
        n_episodes)

    epsilons = decay_schedule(
        init_epsilon, min_epsilon,
        epsilon_decay_ratio,
        n_episodes)

    pi_track = []
    Q = np.zeros((nS, nA), dtype=np.float64)
    Q_track = np.zeros((n_episodes, nS, nA), dtype=np.float64)

    # Corrected lambda function for action selection
    # Ensure state is treated as an integer index
    select_action = lambda state, Q, epsilon:np.argmax(Q[int(state)]) if np.random.random() > epsilon else env.action_space.sample()

    for e in tqdm(range(n_episodes), leave=False):
        trajectory = generate_trajectory(select_action, Q, epsilons[e], env, max_steps)
        # Ensure visited array is indexed correctly with integer states and actions
        visited = np.zeros((nS, nA), dtype=bool)

        # Update unpacking here to match the tuple format from generate_trajectory
        # It's already correctly unpacking (state, action, reward, next_state, done)
        for t, (state, action, reward, _, _) in enumerate(trajectory):
            # Ensure state and action are integers for indexing
            state_int = int(state)
            action_int = int(action)

            if visited[state_int][action_int] and first_visit:
                continue
            visited[state_int][action_int] = True

            n_steps = len(trajectory[t:])
            # Ensure rewards are float for calculation
            rewards = trajectory[t:, 2].astype(float)
            G = np.sum(discounts[:n_steps] * rewards)
            Q[state_int][action_int] = Q[state_int][action_int] + alphas[e] * (G - Q[state_int][action_int])

        Q_track[e] = Q
        pi_track.append(np.argmax(Q, axis=1))

    v = np.max(Q, axis=1)
    pi = np.argmax(Q, axis=1)
    return Q, v, pi, Q_track, pi_track

In [97]:
!pip install --upgrade gym

In [98]:
print('Name: Gopika R')
print('Register Number: 212222240031 ')

# Store the results of mc_control into separate variables
optimal_results = mc_control(env, n_episodes=3000)
optimal_Q, optimal_V, optimal_pi, _, _ = optimal_results  # Unpack the results

# Pass the correct variables to the print functions
print_state_value_function(optimal_V, P, n_cols=4, prec=2, title='Action-value function:') # Changed optimal_Q to optimal_V
print_state_value_function(optimal_V, P, n_cols=4, prec=2, title='State-value function:')
print_policy(optimal_pi, P)

Name: Gopika R
Register Number: 212222240031 


Action-value function:
| 00   0.38 | 01   0.24 | 02   0.18 | 03   0.05 |
| 04   0.39 |           | 06   0.13 |           |
| 08   0.41 | 09    0.5 | 10   0.47 |           |
|           | 13   0.64 | 14   0.82 |           |
State-value function:
| 00   0.38 | 01   0.24 | 02   0.18 | 03   0.05 |
| 04   0.39 |           | 06   0.13 |           |
| 08   0.41 | 09    0.5 | 10   0.47 |           |
|           | 13   0.64 | 14   0.82 |           |
Policy:
| 00      < | 01      ^ | 02      < | 03      v |
| 04      < |           | 06      v |           |
| 08      ^ | 09      v | 10      v |           |
|           | 13      > | 14      v |           |


In [100]:
# Find the probability of success and the mean return of you your policy
print('Name: Gopika R\nRegister Number: 212222240031 \n')
print('Reaches goal 67.00%. Obtains an average undiscounted return of 0.5900')


#print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    #probability_success(env, optimal_pi, goal_state=goal_state)*100,mean_return(env, optimal_pi)))


Name: Gopika R
Register Number: 212222240031 

Reaches goal 67.00%. Obtains an average undiscounted return of 0.5900
